# 신경망 모델 구성하기
- URL from : [PyTorch Tutorial 신경망 모델 구성하기](https://tutorials.pytorch.kr/beginner/basics/buildmodel_tutorial.html)
- 신경망은 데이터에 대한 연산을 수행하는 계층(layer)과 모듈(module)로 구성되어 있다.
- torch.nn에서는 신경망을 구성하는데 필요한 구성 요소를 제공하고 있다. PyTorch의 모든 모듈은 nn.Module의 하위 클래스다.
- 신경망은 다른 모듈, 계층으로 구성된 모듈이다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.

In [1]:
# 필요한 라이브러리를 불러오자
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

학습을 위한 장치를 얻어보자. 가능한 경우엔 GPU와 같은 하드웨어 가속기에서 모델을 학습하는 게 유리하다. torch.cuda를 사용할 수 있는지 확인하고 안 되면 CPU를 계속 사용하게 된다.

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}를 사용할 수 있습니다!")

cuda를 사용할 수 있습니다!


클래스를 정의해보자
- 신경망 모델을 nn.Module의 하위 클래스로 정의하고, __init__에서 신경망 계층들을 초기화한다.
- nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현한다.

In [3]:
class NeuralNetwork(nn.Module) :
    def __init__(self) :
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x) :
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetwork의 인스턴스(instance)를 생성하고 이를 device에 옮겨주고, 구조(structure)를 출력해보자

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


- 모델을 사용하기 위해서 입력 데이터를 전달한다. 이는 일부 백그라운드 연산들과 함께 모델의 forward를 실행한다. model.forward()를 직접 호출하지 않아도 된다.
- 모델에 입력을 호출하면 각 class에 대한 원시(raw) 예측값이 있는 10차원 tensor가 반환된다. 이렇게 반환된 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻게된다.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class : {y_pred} | 예측 클래스는 {y_pred}입니다!")

Predicted class : tensor([6], device='cuda:0') | 예측 클래스는 tensor([6], device='cuda:0')입니다!


모델 계층(Layer)
- FashionMNIST 모델의 계층들을 살펴보자. 이를 보기 위해 28x28 크기의 이미지 3개로 구성된 미니 배치를 가져와서 신경망을 통과할 때 어떤 일이 일어나는지 알아보자.

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten 계층을 초기화해서 각 2D 이미지를 784픽셀(28x28) 값을 갖는 연속된 배열로 변환시켜보자. 이 때, dim=0의 미니 배치의 차원수는 유지된다.

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear 계층은 저장된 가중치(weight)와 편향(bias)을 사용해서 입력 데이터에 선형 변환(linear transformation)을 적용하는 모듈이다.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


- 비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만든다. 비선형 활성화는 선형 변환 후에 적용되어 비선형성(non-linearity)을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.
- 이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화 함수를 도입할 수도 있다.

In [9]:
print(f"Before ReLU : {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU : {hidden1}")

Before ReLU : tensor([[ 0.2199, -0.1440,  0.3022,  0.5243, -0.5172, -0.2134, -0.5256,  0.3203,
          0.3196, -0.3266,  0.0295,  0.0347,  0.3861, -0.3538, -0.3041,  0.3429,
          0.7189,  0.1225, -0.0743, -0.2830],
        [-0.2376, -0.0986,  0.3822,  0.7964, -0.4125, -0.1953, -0.0589,  0.4395,
          0.1655,  0.1001,  0.4564,  0.2645,  0.0458, -0.3512, -0.0416,  0.1713,
          0.5270,  0.1643, -0.0144, -0.0137],
        [-0.1336,  0.0295,  0.0982,  0.9017, -0.2945, -0.0614, -0.0502,  0.2488,
          0.3154, -0.0222,  0.5005,  0.5749, -0.1762, -0.5271, -0.0622,  0.3122,
          0.4012, -0.2462, -0.2006, -0.2985]], grad_fn=<AddmmBackward0>)


After ReLU : tensor([[0.2199, 0.0000, 0.3022, 0.5243, 0.0000, 0.0000, 0.0000, 0.3203, 0.3196,
         0.0000, 0.0295, 0.0347, 0.3861, 0.0000, 0.0000, 0.3429, 0.7189, 0.1225,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.3822, 0.7964, 0.0000, 0.0000, 0.0000, 0.4395, 0.1655,
         0.1001, 0.4564, 0.2645, 0.0458, 0.0000, 0.

nn.Sequential은 순서를 갖는 모듈의 컨테이너라고 볼 수 있다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다. 순차 컨테이너(sequential container)를 사용해서 아래의 seq_modules와 같은 신경망을 만들어낼 수 있다.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

- 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 [-infty, infty] 범위의 원시 값(raw value)인 logits를 반환한다.
- logits는 모델의 각 class에 대한 예측 확률을 나타내도록 [0,1] 범위로 비례해서 조정된다.
- dim 매개변수는 값의 합이 1이 되는 차원을 나타낸다.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab= softmax(logits)

모델 매개변수
- 신경망 내부의 많은 계층들은 매개변수화(parameterize) 된다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다. nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 된다.
- 본 예제에서는 각 매개변수들을 순회(iterate)하며, 매개변수의 크기와 값을 출력해보자.

In [13]:
print(f"Model structure : {model}\n\n")
for name, param in model.named_parameters() :
    print(f"Layer : {name} | Size : {param.size()} | Values : {param[:2]} \n")

Model structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[-0.0329,  0.0067,  0.0260,  ..., -0.0318,  0.0129,  0.0164],
        [ 0.0156, -0.0286,  0.0066,  ...,  0.0214, -0.0209, -0.0329]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([-0.0252, -0.0303], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[ 0.0232, -0.0239, -0.0217,  ..., -0.0344, -0.0239, -0.0110],
        [-0.0037,  0.0278, -0.0331,  ...,  0.0209, -0.0100, -0.0280]],
       device='cuda:0', grad